In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

# ----- Step 1: Create synthetic images and labels -----
os.makedirs("images", exist_ok=True)
n_images = 50
img_size = 64
labels = []

for i in range(n_images):
    img = np.zeros((img_size, img_size), dtype=np.uint8)
    x1, y1 = np.random.randint(10, 40), np.random.randint(10, 40)
    w, h = np.random.randint(10, 20), np.random.randint(10, 20)
    x2, y2 = x1 + w, y1 + h
    cv2.rectangle(img, (x1, y1), (x2, y2), 255, -1)
    cv2.imwrite(f"images/image_{i}.png", img)
    labels.append([i, x1, y1, x2, y2, "settlement"])

# Save labels to CSV
df = pd.DataFrame(labels, columns=['image_id', 'x_min', 'y_min', 'x_max', 'y_max', 'class'])
df.to_csv("labels.csv", index=False)

# ----- Step 2: Dataset and Dataloader -----
class SettlementDataset(Dataset):
    def __init__(self, image_dir, labels_df, transform=None):
        self.image_dir = image_dir
        self.labels_df = labels_df
        self.transform = transform

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        label = self.labels_df.iloc[idx]
        image_path = os.path.join(self.image_dir, f"image_{label.image_id}.png")
        image = Image.open(image_path).convert("L")
        bbox = torch.tensor([label.x_min, label.y_min, label.x_max, label.y_max], dtype=torch.float32)
        if self.transform:
            image = self.transform(image)
        return image, bbox

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

df = pd.read_csv("labels.csv")
dataset = SettlementDataset("images", df, transform=transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# ----- Step 3: Simple CNN Model -----
class SettlementDetector(nn.Module):
    def __init__(self):
        super(SettlementDetector, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 8, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(8, 16, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(16 * 16 * 16, 128), nn.ReLU(),
            nn.Linear(128, 4)  # x_min, y_min, x_max, y_max
        )

    def forward(self, x):
        x = self.conv(x)
        return self.fc(x)

# ----- Step 4: Train the Model -----
model = SettlementDetector()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print("Training...")
for epoch in range(10):
    for images, targets in dataloader:
        preds = model(images)
        loss = criterion(preds, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} | Loss: {loss.item():.4f}")

# ----- Step 5: Test on a Sample -----
import matplotlib.pyplot as plt

sample_img, true_box = dataset[0]
model.eval()
with torch.no_grad():
    pred_box = model(sample_img.unsqueeze(0)).squeeze().numpy()

img_np = sample_img.squeeze().numpy() * 255
img_np = img_np.astype(np.uint8)
img_color = cv2.cvtColor(img_np, cv2.COLOR_GRAY2BGR)
true_box = true_box.numpy().astype(int)
pred_box = pred_box.astype(int)

cv2.rectangle(img_color, tuple(true_box[:2]), tuple(true_box[2:]), (0, 255, 0), 1)  # Green = True
cv2.rectangle(img_color, tuple(pred_box[:2]), tuple(pred_box[2:]), (0, 0, 255), 1)  # Red = Pred

plt.imshow(img_color)
plt.title("Green = True, Red = Predicted")
plt.axis("off")
plt.show()
